In [1]:
from utility import *
import pandas_ta as ta

In [2]:
data = get_yf_data()

[*********************100%***********************]  1 of 1 completed


In [5]:
# Adding a simple moving average
data['SMA'] = ta.sma(data['Close'], length=14)
data

,Open,High,Low,Close,Adj Close,Volume,SMA
Date,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.876099,135480400,NaN
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.167603,146322800,NaN
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.742661,118387200,NaN
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.400551,108872000,NaN
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.565201,132079200,NaN
...,...,...,...,...,...,...,...
2022-12-23,130.919998,132.419998,129.639999,131.860001,130.631363,63814900,138.360715
2022-12-27,131.380005,131.410004,128.720001,130.029999,128.818405,69007800,137.440714
2022-12-28,129.669998,131.029999,125.870003,126.040001,124.865585,85438400,136.376428


In [4]:
# Assuming 'data' is your dataframe with historical stock prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Prepare data for LSTM (e.g., 60 timesteps to predict the next price)
def prepare_data(dataset, timestep):
    X, y = [], []
    for i in range(timestep, len(dataset)):
        X.append(dataset[i-timestep:i, 0])
        y.append(dataset[i, 0])
    return np.array(X), np.array(y)

timestep = 60
X_train, y_train = prepare_data(scaled_data, timestep)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10)
